# Chapter 10: Create Your Own Game Environment

In this chapter, you’ll learn how to create your own game environments so that you can use deep learning to develop game strategies and play games in them. You'll also use these game environments to practice Reinforcement Learning and Deep Q Learning game strategies later in this book.

Since you are already familiar with the Frozen Lake game from the OpenAI Gym, You’ll learn to create your own Frozen Lake game environment, with all the attributes and methods as the one in OpenAI Gym. Better yet, you'll add a graphical game board using the ***turtle*** library so that you can visualize the state you are in as the game progresses. Along the way, you’ll learn the necessary skills to create a game environment, an agent, and how the agent interacts with the environment. You’ll code in how to change from one state to another based on agent’s actions, how to determine rewards, and how to determine if the game has ended. Later in this book, you'll use these skills to create game environments for Tic Tac Toe and Connect Four. 
 
Finally, you'll learn how to save a ***turtle*** game board as a png file on your computer. You'll save all game boards in a full game, then convert them into an animation, as follows:
<img src="https://gattonweb.uky.edu/faculty/lium/ml/frozen_steps.gif" />

***
$\mathbf{\text{Create a subfolder for files in Chapter 10}}$<br>
***
We'll put all files in Chapter 10 in a subfolder /files/ch10. Run the code in the cell below to create the subfolder.

***

In [1]:
import os

os.makedirs("files/ch10", exist_ok=True)

***
$\mathbf{\text{Install needed modules for Chapter 10}}$<br>
***
To covert a ps file to a png file, you need to conda install Ghostscript. We'll use it in this chapter and many later chapters. 

Enter the following command in the Anaconda prompt (Windows) or the terminal (MAC/Linux) to activate the virtual environment that you have created for this book animatedML

`conda activate animatedML`

then enter the following command in the same Anaconda prompt or terminal:

`conda install -c conda-forge ghostscript`

Follow the on-screen instructions to finish the installation. 

After that, restart you Jupyter Notebook for it to take effect.

***

## 1. Create the Frozen Lake Environment
We have played the Frozen Lake game in the OpenAI Gym environment, without a graphical game 
board. We’ll create such an environment from scratch, with all the features and methods as in the OpenAI Gym environment and add graphical game boards to it. 

### 1.1. Use A Python Class to Represent the Environment
We’ll create a Python class to represent the Frozen Lake environment. The class will have various attributes, variables, and methods to replicate those in the Frozen Lake game in OpenAI Gym. 

#### Game Attributes

The self-made Frozen Lake game will have the following attributes:
 
*	action_space: an attribute that provides the space of all actions that can be taken by the agent.
*	observation_space: an attribute that provides the list of all possible states in the environment.
*	state: an attribute indicating which state the agent is currently in.
*	action: an attribute indicating the action taken by the agent.
*	reward: is an attribute indicating the reward to the agent because of the action taken by the agent.
*	done: an attribute indicating whether the game has ended.
*	info: an attribute that provides information about the game.

#### Game Methods

The self-made Frozen Lake game will have a few methods as well:
 
*	reset() is a method to set the game environment to the initial (that is, the starting) state.
*	render() is a method showing the current state of the environment graphically.
*	step() is a method that returns the new state, the reward, the value of *done* variable, and the variable *info* based on the action taken by the agent.
*	sample() is a method to randomly choose an action from the action space.
*	close() is a method to end the game environment.

### 1.2. Create A Local Package
We'll create a local package for this book to contain all self-made local modules. We'll call the package ***utils***. 

A Python module is a single file with the .py extension. In contrast, a
Python package is a collection of Python modules contained in a single
directory. The directory must have a file named *__init__.py* to distinguish
it from a directory that happens to have .py extension files in it.
I’ll guide you through the process of creating a local package
step-by-step. 

The code below will create the local package.

In [ ]:
os.makedirs("utils", exist_ok=True)
outfile = open('utils/__init__.py', 'a')
outfile.close()

We first create a local directory *utils*. We then create an empty file *__init__.py* inside the directory. This way, the directory *utils* becomes a local package, not just a local folder. 

Later, we'll gradually put more modules inside the package.

Now let's code in a self-made Frozen Lake game environment using a Python class. Save the code in the cell below as *frozenlake_env.py* in the folder *utils* you just created. Alternatively, you can download it from my GitHub repository. 

In [2]:
import turtle as t
import numpy as np
import time


# Define an action_space helper class
class action_space:
    def __init__(self, n):
        self.n = n
    def sample(self):
        return np.random.choice(range(self.n))
    
# Define an obervation_space helper class    
class observation_space:
    def __init__(self, n):
        self.shape = (n,)

# Define the Frozen game environment
class Frozen():
    def __init__(self):        
        self.done=False
        self.reward=0.0
        self.info={'prob': 1.0}
        self.state=0
        self.actual_actions=[(0, -1), (1, 0), (0, 1), (-1, 0)]
        self.action_space=action_space(4)
        self.observation_space=observation_space(16)
        self.showboard=False  
        self.grid=[]
        for x in range(4):
            for y in range(4):
                self.grid.append((x,y))

    def reset(self):  
        self.state=0
        self.done=False
        self.reward=0.0
        return self.state

    def step(self, action):
        actual_action=self.actual_actions[action]
        co_or=self.grid[self.state]
        new_co_or=(actual_action[0]+co_or[0], actual_action[1]+co_or[1])
        if actual_action[0]+co_or[0]<0 or actual_action[0]+co_or[0]>3:
            new_co_or=co_or
        if actual_action[1]+co_or[1]<0 or actual_action[1]+co_or[1]>3:
            new_co_or=co_or        
        new_state=self.grid.index(new_co_or)
        if new_state==15:
            self.reward=1.0
            self.done=True
        if new_state in [5,7,11,12]:
            self.reward=-1.0
            self.done=True
        self.state=new_state
        return new_state, self.reward, self.done, self.info
 
    def display_board(self):
        try:
            t.setup(850,850, 10, 70) 
        except t.Terminator:
            t.setup(850,850, 10, 70)                       
        t.hideturtle()
        t.bgcolor("alice blue")
        t.tracer(False)
        t.title('The Frozen Lake Game')
        t.clear()
        t.pensize(3)
        for i in range(-400,600,200):  
            t.up()
            t.goto(i, -400)
            t.down()
            t.goto(i, 400)
            t.up()
            t.goto(-400,i)
            t.down()
            t.goto(400,i)
            t.up()           
        for (x,y) in [(-100,100),(300,100),(300,-100),(-300,-300)]:
            t.goto(x,y)
            t.dot(150,"light gray")
            t.color('blue')
            t.goto(x-30,y-20)
            t.write("hole",font=("Helvetica",30)) 
        self.xycor={(3, 0):(-400, -400), (2, 0):(-400, -200), 
               (1, 0):(-400, 0), (0, 0):(-400, 200), 
               (3, 1):(-200, -400), (2, 1):(-200, -200), 
               (1, 1):(-200, 0), (0, 1):(-200, 200), 
               (3, 2):(0, -400), (2, 2):(0, -200), 
               (1, 2):(0, 0), (0, 2):(0, 200), 
               (3, 3):(200, -400), (2, 3):(200, -200), 
               (1, 3):(200, 0), (0, 3):(200, 200)}
        state_grid = self.grid[self.state]
        state_xy = self.xycor[state_grid]
        t.color('blue')
        t.goto(-350,350)
        t.write("start",font=("Helvetica",30))        
        t.goto(300,-350)
        t.write("goal",font=("Helvetica",30)) 
        t.color('black')
        # Place a red dot in the current state
        self.fall = t.Turtle()
        self.fall.up()
        self.fall.hideturtle()
        self.fall.goto(state_xy[0]+100,state_xy[1]+100)
        self.fall.dot(150,"red")
        t.update()

    def render(self):
        if self.showboard==False:
            self.display_board()
            self.showboard=True        
        state_grid = self.grid[self.state]
        state_xy = self.xycor[state_grid]
        # update the current position
        self.fall.clear()
        self.fall.goto(state_xy[0]+100,state_xy[1]+100)
        self.fall.dot(150,"red")
        t.update()
    def close(self):
        time.sleep(1)
        try:
            t.bye()
        except t.Terminator:
            print('exit turtle')

If you run the above cell, nothing will happen. The class simply creates a game environment. We need to initiate the game environment and start playing using Python programs, just as you do with an OpenAI Gym game environment. We'll do that in the next subsection.

### 1.3. Verify the Custom-Made Game Environment
Next, we'll check the attributes and methods of the self-made game environment and make sure it has all the elements that are provided by the OpenAI Gym. What you'll do is to use the old programs in Chapter 8.

In Chapter 8, you used the code below to initiate the game environment:


```python
import gym 
 
env = gym.make("CartPole-v0")
env.reset()                    
env.render()
```

We'll do something similar. Run the code in the cell below.

In [2]:
from utils.frozenlake_env import Frozen

env = Frozen()
env.reset()                    
env.render()

You should see a separate turtle window, with a game board as follows: 
<img src="https://gattonweb.uky.edu/faculty/lium/ml/frozen0.png" />

If you want to close the game board window, use the *close()* method, like so:

In [3]:
env.close()

Next, we'll check the attributes of the environment such as the observation space and action space. 

In [4]:
env=Frozen()
# check the action space
number_actions = env.action_space.n
print("the number of possible actions are", number_actions)
# sample the action space ten times
print("the following are ten sample actions")
for i in range(10):
   print(env.action_space.sample())
# check the shape of the observation space
print("the shape of the observation space is", env.observation_space.shape)

the number of possible actions are 4
the following are ten sample actions
1
1
2
1
0
2
3
1
1
1
the shape of the observation space is (16,)


The meanings of the actions in this game are exactly the same as those in the OpenAI Gym Frozen Lake game:
* 0: Going left
* 1: Going down
* 2: Going right
* 3: Going up


The state space has 16 values: 0, 1, 2, …, 15. The top left square is state 0, the top right is state 3, and the bottom right corner is 15, as shown in the following picture:
<img src="https://gattonweb.uky.edu/faculty/lium/ml/lakesurface.png" />

## 2. Play Games in the Custom-Made Environment
Next, we'll play games in the custom-made environment. You'll learn to save each game board as a picture. Finally, you'll record all game boards in a full game, and convert them into an animation.

### 2.1. Play a full game

Here we'll play a full game, by randomly choosing an action from the action space each step.

In [5]:
import time
# Initiate the game environment
env=Frozen()
state=env.reset()   
env.render()
# Play a full game
while True:
    # randomly choose an action
    action = env.action_space.sample()
    print(f"the current state is state={state}")
    print(f"the current action is action={action}")    
    new_state, reward, done, info = env.step(action)
    print(f"new state={new_state}, reward={reward}, done={done}")
    # Current new state becomes the state in the next round
    state=new_state
    env.render()
    # slow down the game pace by stopping for one second
    time.sleep(1)
    # Stop the game if done is True
    if done == True:
        if state==15:
            print("Congrats, you won the game!")
        else:
            print("Better luck next time!")
        break
env.close()        

the current state is state=0
the current action is action=3
new state=0, reward=0.0, done=False
the current state is state=0
the current action is action=2
new state=1, reward=0.0, done=False
the current state is state=1
the current action is action=2
new state=2, reward=0.0, done=False
the current state is state=2
the current action is action=1
new state=6, reward=0.0, done=False
the current state is state=6
the current action is action=2
new state=7, reward=-1.0, done=True
Better luck next time!


Note that the outcome is different each time you run it because the actions are randomly chosen.

### 2.2. Play the Game Manually
Next, you’ll learn how to manually interact with the Frozen Lake game, just like you did in Chapter 8. The following lines of code show you how.

In [6]:
# Initiate the game environment
env=Frozen()
state=env.reset()   
env.render()   

print('''
enter 0 for left, 1 for down
2 for right, and 3 for up
''')

while True:
    print(f"the current state is state={state}")
    try:
        action = int(input('how do you want to move?\n'))
    except:
        print('please enter 0, 1, 2, or 3')
    print(f"you have chosen action={action}")
    new_state, reward, done, _ = env.step(action)
    state = new_state
    env.render()
    if done==True:
        if new_state==15:
            print("Congrats, you have made to the destination!")
        else:
            print("Game over. Better luck next time!")
        break  
env.close()


enter 0 for left, 1 for down
2 for right, and 3 for up

the current state is state=0
how do you want to move?
1
you have chosen action=1
the current state is state=4
how do you want to move?
1
you have chosen action=1
the current state is state=8
how do you want to move?
2
you have chosen action=2
the current state is state=9
how do you want to move?
2
you have chosen action=2
the current state is state=10
how do you want to move?
1
you have chosen action=1
the current state is state=14
how do you want to move?
2
you have chosen action=2
Congrats, you have made to the destination!


### 2.3. Save the Current Game Board as A Picture
Next, you'll learn how to save the current game board as a picture on your computer.

In [8]:
import turtle as t

env=Frozen()
env.reset()                    
# use the getscreen() method in turtle to record screen
try:
    ts = t.getscreen()
except t.Terminator:
    ts = t.getscreen()   
env.render()
# save the screen as a ps file
ts.getcanvas().postscript(file=f"files/ch10/frozent.ps")
time.sleep(5)
env.close()    

***
$\mathbf{\text{A known Terminator error in the turtle library}}$<br>
***
It's a known problem that turtle windows may not close properly. As a result, it gives you an error message every other time you run it. I used the exception handling above to address the issue. By initiating the ***turtle*** library twice, we can avoid the error message here.
***


After that, run the following cell to save the ps files on your computer.

In [6]:
import matplotlib
from PIL import Image

image = Image.open("files/ch10/frozent.ps")
pngfigure = image.convert('RGBA')
pngfigure.save("files/ch10/frozent.png", lossless = True)

Now, if you open the file frozent.png in your local folder, you'll see the following picture:
<img src="https://gattonweb.uky.edu/faculty/lium/ml/frozent.png" />

### 2.4. Animate A Full Game

Now that you know how to record the board position at any moment, you can record all the game positions in a full game sequentially, and make an animation out of it, like so:

In [9]:
import turtle as t

env = Frozen()
env.reset()   

print('''
enter 0 for left, 1 for down
2 for right, and 3 for up
''')

step = 1
try:
    ts = t.getscreen()
except t.Terminator:
    ts = t.getscreen()    
env.render()
ts.getcanvas().postscript(file=f"frozen0.ps")
while True:

    try:
        action = int(input('how do you want to move?\n'))
    except:
        print('please enter 0, 1, 2, or 3')
    new_state, reward, done, info = env.step(action)
    env.render()
    ts.getcanvas().postscript(file=f"frozen{step}.ps")
    step += 1
    if done==True:
        if new_state==15:
            print("Congrats, you have made to the destination!")
        else:
            print("Game over. Better luck next time!")
        break  

env.close()

from PIL import Image

for i in range(step):
    im = Image.open(f"frozen{i}.ps")
    fig = im.convert('RGBA')
    fig.save(f"frozen{i}.png", lossless = True)

import numpy as np
import imageio
frames=[]
for i in range(step):
    im = Image.open(f"frozen{i}.ps")
    frame=np.asarray(im)
    frames.append(frame) 
imageio.mimsave('frozen_steps.gif', frames, fps=1) 


enter 0 for left, 1 for down
2 for right, and 3 for up

how do you want to move?
1
how do you want to move?
1
how do you want to move?
2
how do you want to move?
2
how do you want to move?
1
how do you want to move?
2
Congrats, you have made to the destination!


I used six steps to finish the game. All six steps, plus the initial screen, are all captured and saved as png files in the chapter folder. For example, the last position of the board is frozen6.png, and it looks as follows:
<img src="https://gattonweb.uky.edu/faculty/lium/ml/frozen6.png" />

It shows that I have successfully reached the goal and won the game.

Finally, I have also combined the seven pictures into an animation in the gif format. If you open the file frozen_steps.gif in your chapter folder, you should see the following:
<img src="https://gattonweb.uky.edu/faculty/lium/ml/frozen_steps.gif" />

## 3. Play the Game with the Trained Model
We have already trained a deep neural network model to play the Frozen Lake game in the OpenAI Gym environment. Since our self-made game environment has all the features, plus the graphical game board, we can play the game with the trained model from Chapter 8. 

In [10]:
# import needed modules
import numpy as np
import tensorflow as tf
from utils.frozenlake_env import Frozen

# note that model is saved in the local folder for Chapter 8
reload = tf.keras.models.load_model("files/ch08/trained_frozen.h5")

# Define a onehot_encoder() function
def onehot_encoder(value, length):
    onehot=np.zeros((1,length))
    onehot[0,value]=1
    return onehot

action0=onehot_encoder(0, 4)
action1=onehot_encoder(1, 4)
action2=onehot_encoder(2, 4)
action3=onehot_encoder(3, 4)

env = Frozen()
state = env.reset()   

while True:
    # Convert state and action into onehots 
    state_arr = onehot_encoder(state, 16)
    # Use the trained model to predict the prob of winning 
    sa0 = np.concatenate([state_arr, action0], axis=1)    
    sa1 = np.concatenate([state_arr, action1], axis=1)  
    sa2 = np.concatenate([state_arr, action2], axis=1)  
    sa3 = np.concatenate([state_arr, action3], axis=1)
    sa = np.concatenate([sa0, sa1, sa2, sa3], axis=0)
    action = np.argmax(reload.predict(sa, verbose=0))
    print(action)
    new_state, reward, done, info = env.step(action)
    env.render()
    print(new_state, reward, done, info) 
    state = new_state
    if done == True:
        break
env.close()

1/1 [==============================] - 0s 318ms/step
1
4 0.0 False {'prob': 1.0}
1/1 [==============================] - 0s 16ms/step
1
8 0.0 False {'prob': 1.0}
1/1 [==============================] - 0s 15ms/step
2
9 0.0 False {'prob': 1.0}
1/1 [==============================] - 0s 14ms/step
1
13 0.0 False {'prob': 1.0}
1/1 [==============================] - 0s 14ms/step
2
14 0.0 False {'prob': 1.0}
1/1 [==============================] - 0s 15ms/step
2
15 1.0 True {'prob': 1.0}


The player wins the game with the shortest possible path. So the deep learning game strategy works in the self-made environment as well!!!